# Python Tips 6

author@snowhyzhang

- [扁平化列表](#flatten_list)
- [带可变状态的生成器](#state_iter)
- [对迭代器做切片](#islice)

---
<a name='faltten_list'>

## 扁平化列表

通过使用`yield from`语句结合递归的方法可以方便地实现扁平化列表

In [1]:
def flatten_list(l):
    for e in l:
        if isinstance(e, list):
            yield from flatten_list(e)
        else:
            yield e

l = [[1, 2, 3, [4, 5]], [6, 7, [8, 9]], 10]
for e in flatten_list(l):
    print(e, end=' ')

1 2 3 4 5 6 7 8 9 10 

`yield from`是`Python`3.3开始出现的语法，其与`yield`的区别在于`yield from`可以将后面的可迭代对象中的每个元素一个一个的返回，而`yiled`则会返回整个可迭代对象，列如

In [2]:
l = [[1, 2, 3], [4, 5, 6]]

def test_yield(l):
    for e in l:
        yield e

def test_yield_from(l):
    for e in l:
        yield from e
        
for e in test_yield(l):
    print(e, end=' ')

print()
for e in test_yield_from(l):
    print(e, end=' ')

[1, 2, 3] [4, 5, 6] 
1 2 3 4 5 6 

---
<a name='state_iter'>

## 带可变状态的生成器

如果我们想要定义一个生成器，又希望能够改变该生成器中一些参数，例如我们定义一个返回`batch_size`数量的元素，其中，`batch_size`是可以通过外部随时更改的，如果使用生成器函数定义，那会导致相当复杂的代码。这时，我们可以通过定义一个类，并且实现类中的`__iter__`方法即可

In [3]:
class BatchIter:
    def __init__(self, data, batch_size=3):
        self.data = data
        self.batch_size = batch_size
        
    def __iter__(self):
        size = 0
        results = []
        for e in self.data:
            results.append(e)
            size = size + 1
            if size == self.batch_size:
                yield results
                results = []
                size = 0
        if results:
            yield results

我们希望每次返回一个\[1, 3\]之间随机数量的元素，我们可以通过更改对象`batch_size`的方法来实现

In [4]:
import random

batch_size = random.randint(1, 3)
batch_iter = BatchIter(range(20), batch_size)
for batch in batch_iter:
    print('batch_size:{}, result: {}'.format(batch_size, batch))
    batch_size = random.randint(1, 3)
    batch_iter.batch_size = batch_size

batch_size:2, result: [0, 1]
batch_size:2, result: [2, 3]
batch_size:3, result: [4, 5, 6]
batch_size:2, result: [7, 8]
batch_size:3, result: [9, 10, 11]
batch_size:1, result: [12]
batch_size:1, result: [13]
batch_size:1, result: [14]
batch_size:1, result: [15]
batch_size:3, result: [16, 17, 18]
batch_size:1, result: [19]


---
<a name='islice'>

### 迭代器做切片

普通的切片操作对迭代器并不起作用

In [5]:
# 定义一个简单的生成器函数
def my_iter(l):
    for e in l:
        yield e

it = my_iter([1, 2, 3, 4, 5])
# ERROR
# it[1:3]

这时，我们可以使用`itertools`模块中的`islice`函数，`islice`将会产生一个切片范围内的迭代器。

In [6]:
from itertools import islice

for e in islice(it, 1, 3):
    print(e)

2
3


不过需要注意的是，`islice`会消耗原迭代器的数据

In [7]:
# 对it调用next的结果会是切片后的数值，即4
next(it)

4